In [1]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from utils import *
from kinect_image_conversion import *
import cv2 as cv

DETECT_PARAM = 300

In [2]:
generateKinectDataset()

In [3]:
IMG_DIR = './Images'

color_image = Image.open(IMG_DIR + "/rgb_image_2.png")
depth_image = Image.open(IMG_DIR + "/depth_image_2.png").convert("L")

scale_factor = scaleCalibration(color_image, depth_image, DETECT_PARAM)

Press ENTER to accept the detection...
Press ENTER to accept the detection...
1.127659574468085


In [4]:
# RESIZE ALL IMAGES AFTER EXTRINSIC CALIBRATION
IMG_DIR = './Images'
files_num = len([name for name in os.listdir(IMG_DIR) if os.path.isfile(os.path.join(IMG_DIR, name))])

for i in range(files_num // 2):
    path = IMG_DIR + "/rgb_image_" + str(i + 2) + ".png"
    rgb_image = cv.imread(path)

    original_dimensions = (rgb_image.shape[1], rgb_image.shape[0])

    new_width = int(rgb_image.shape[1] * scale_factor)
    new_height = int(rgb_image.shape[0] * scale_factor)
    new_dimensions = (new_width, new_height)

    scaled_image = cv2.resize(rgb_image, new_dimensions, interpolation=cv2.INTER_CUBIC)

    x_start = (new_width - original_dimensions[0]) // 2
    y_start = (new_height - original_dimensions[1]) // 2
    cropped_image = scaled_image[y_start:y_start + HEIGHT, x_start:x_start + WIDTH]

    cv2.imwrite(path, cropped_image)

IMG_DIR = './Images/Calibration'

for i in range(2):
    path = IMG_DIR + "/rgb_image_" + str(i) + ".png"
    rgb_image = cv.imread(path)

    original_dimensions = (rgb_image.shape[1], rgb_image.shape[0])

    new_width = int(rgb_image.shape[1] * scale_factor)
    new_height = int(rgb_image.shape[0] * scale_factor)
    new_dimensions = (new_width, new_height)

    scaled_image = cv2.resize(rgb_image, new_dimensions, interpolation=cv2.INTER_CUBIC)

    x_start = (new_width - original_dimensions[0]) // 2
    y_start = (new_height - original_dimensions[1]) // 2
    cropped_image = scaled_image[y_start:y_start + HEIGHT, x_start:x_start + WIDTH]

    cv2.imwrite(path, cropped_image)


In [5]:
# INTRINSIC CALIBRATION PROCESS
calibration = []

for i in range(2):
    with open("./Kinecet_Images_Measured/_measures.txt", "r") as file:
        for j, line in enumerate(file):
            if j == i:
                calib_distance = int(line.strip())
    calib_data = (Image.open("Images/Calibration/rgb_image_" + str(i) + ".png"), Image.open("Images/Calibration/depth_image_" + str(i) + ".png").convert("L"), calib_distance)
    calibration.append(calib_data)

int_calib = distanceCalibration(calibration[0][0], calibration[1][0], calibration[0][1], calibration[1][1], calibration[0][2], calibration[1][2], DETECT_PARAM)

Press ENTER to accept the detection...
Press ENTER to accept the detection...
6.090909090909093 237.90909090909088


In [8]:
from PIL import ImageDraw, ImageFont

IMG_DIR = './Images'
files_num = len([name for name in os.listdir(IMG_DIR) if os.path.isfile(os.path.join(IMG_DIR, name))])

for i in range(files_num // 2):
    color_image = Image.open(os.path.join(IMG_DIR, f"rgb_image_{i + 2}.png"))
    depth_image = Image.open(os.path.join(IMG_DIR, f"depth_image_{i + 2}.png")).convert("L")

    x, y, _ = detectReference(color_image, DETECT_PARAM)
    dist = calculateDistance(depth_image, (x, y), int_calib)

    draw = ImageDraw.Draw(color_image)

    label = f"{dist:.1f} cm"

    point_radius = 5
    draw.ellipse((x - point_radius, y - point_radius, x + point_radius, y + point_radius), fill="red")

    text_x, text_y = x + 10, y - 10

    font = ImageFont.load_default()

    draw.text((text_x, text_y), label, fill="yellow", font=font)

    output_path = os.path.join("Images/Results/", f"rgb_image_{i + 2}.png")
    color_image.save(output_path)

    print(f"Object {str(i + 2)} is distant {dist}cm")


Press ENTER to accept the detection...
Object 2 is distant 116.09090909090908cm
Press ENTER to accept the detection...
Object 3 is distant 150.63636363636363cm
Press ENTER to accept the detection...
Object 4 is distant 136.09090909090907cm
Press ENTER to accept the detection...
Object 5 is distant 70.63636363636364cm
Press ENTER to accept the detection...
Object 6 is distant 105.18181818181817cm
